In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature_pressure import*

[12:02:57] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[12:02:59] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0.05
batch_size = 128

In [3]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()

In [4]:
import pandas as pd
density_database = pd.read_excel(os.path.join(root_path,'datasets/il properties/density_P.xlsx'),sheet_name='raw')

In [5]:
train_IL_smiles = density_database['IL SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_T =density_database['normalized_T']
train_P = density_database['normalized_P']
train_lngamma = density_database['ρ / kg/m3']
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_IL_smiles,train_T,train_P,train_lngamma))
train_dataloader = get_train_dataloader(train_dataset,batch_size=batch_size)

In [ ]:
epochs, learning_rate = 150,0.001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
max_len = 100
save_name = 'density_best.params'
train(net, train_dataloader, batch_size, learning_rate, ctx, epochs,save_name=save_name)